![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [90]:
import pandas as pd
import numpy as np

# Start coding here...

In [91]:
df = pd.read_csv("bank_marketing.csv")

for col in ["credit_default", "mortgage", "previous_outcome", "campaign_outcome"]:
    print(col)
    print("--------------")
    print(df[col].value_counts())

credit_default
--------------
no         32588
unknown     8597
yes            3
Name: credit_default, dtype: int64
mortgage
--------------
yes        21576
no         18622
unknown      990
Name: mortgage, dtype: int64
previous_outcome
--------------
nonexistent    35563
failure         4252
success         1373
Name: previous_outcome, dtype: int64
campaign_outcome
--------------
no     36548
yes     4640
Name: campaign_outcome, dtype: int64


In [92]:
df.shape

(41188, 16)

In [93]:
df.columns

Index(['client_id', 'age', 'job', 'marital', 'education', 'credit_default',
       'mortgage', 'month', 'day', 'contact_duration', 'number_contacts',
       'previous_campaign_contacts', 'previous_outcome', 'cons_price_idx',
       'euribor_three_months', 'campaign_outcome'],
      dtype='object')

In [94]:
client=df[["client_id", "age", "job", "marital", "education", "credit_default","mortgage"]]
client['job']=client['job'].str.replace('.','_')
client['education']=client['education'].str.replace('.','_')
cDefault=client['credit_default']=='yes'
client.loc[cDefault,'credit_default']=1
client.loc[~cDefault,'credit_default']=0
mortgage=client['mortgage']=='yes'
client.loc[mortgage,'mortgage']=1
client.loc[~mortgage,'mortgage']=0
unknown=client['education']=="unknown"
client.loc[unknown,'education']=np.NAN
client["credit_default"]=client["credit_default"].astype("boolean")
client["mortgage"]=client["mortgage"].astype("boolean")
client.dtypes

client_id           int64
age                 int64
job                object
marital            object
education          object
credit_default    boolean
mortgage          boolean
dtype: object

In [95]:
# Ensure 'day' and 'month' columns are included in the DataFrame selection
campaign = df[["client_id", "number_contacts", "contact_duration", "previous_campaign_contacts", "previous_outcome", "campaign_outcome"]]
campaign["last_contact_date"] = pd.to_datetime(df['day'].astype(str) + "-" + df['month'].astype(str) + "-2022", infer_datetime_format=True, errors='coerce')
campaign["last_contact_date"]=campaign["last_contact_date"].dt.strftime("%Y-%m-%d")
success=campaign['previous_outcome']=='success'
campaign.loc[success,'previous_outcome']=1
campaign.loc[~success,'previous_outcome']=0
yes=campaign['campaign_outcome']=='yes'
campaign.loc[yes,'campaign_outcome']=1
campaign.loc[~yes,'campaign_outcome']=0
#print(success)
#campaign.previous_outcome.sum()
#campaign.campaign_outcome.sum()
campaign['campaign_outcome']=campaign['campaign_outcome'].astype('boolean')
campaign['previous_outcome']=campaign['previous_outcome'].astype('boolean')
campaign['last_contact_date']=pd.to_datetime(campaign['last_contact_date'])
campaign.dtypes

client_id                              int64
number_contacts                        int64
contact_duration                       int64
previous_campaign_contacts             int64
previous_outcome                     boolean
campaign_outcome                     boolean
last_contact_date             datetime64[ns]
dtype: object

In [96]:
economics=df[['client_id','cons_price_idx','euribor_three_months']]
economics.head()
economics.dtypes

client_id                 int64
cons_price_idx          float64
euribor_three_months    float64
dtype: object

In [97]:
client.head()

,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,housemaid,married,basic_4y,False,False
1,1,57,services,married,high_school,False,False
2,2,37,services,married,high_school,False,True
3,3,40,admin_,married,basic_6y,False,False
4,4,56,services,married,high_school,False,False


In [98]:
client.to_csv("client.csv",index=False)
campaign.to_csv("campaign.csv",index=False)
economics.to_csv("economics.csv",index=False)